# Data for reproducibility of `3dq8_20M`

We provide a tutorial on how to load metadata and numerical fits to reproduce our results and/or producing your own fit to the same data used for the `3dq8_20M` model.

In [1]:
import pandas as pd
import numpy as np
import joblib

## Load metadata

We load metadata from the public SXS catalog of binary black holes (with data up to Aug 2024)

In [2]:
filename = '../postmerger/data/waveform_params/SXS_params.csv'
df = pd.read_csv(filename)
df.tail()

,waveform_id,mass1,mass2,mass_ratio,spin1,chi1x,chi1y,chi1z,spin2,chi2x,...,chi2z,alpha,beta,gamma,massf,spinf,chifx,chify,chifz,eccentricity
2014,SXS:BBH:2160,0.750017,0.250000,3.000070,0.600044,-7.643754e-08,1.709403e-07,0.600044,0.399866,-9.215039e-08,...,-0.399866,3.141592e+00,3.118582e-07,3.141592e+00,0.958900,0.785484,-1.338528e-08,1.751252e-07,0.785484,0.000180
2015,SXS:BBH:2161,0.750023,0.249996,3.000142,0.599951,5.996321e-07,-4.469218e-08,0.599951,0.000006,2.309260e-09,...,0.000006,3.552439e-02,1.002262e-06,3.552433e-02,0.957052,0.796406,5.573484e-07,2.401980e-07,0.796406,0.000164
2016,SXS:BBH:2162,0.750016,0.249999,3.000073,0.600050,-8.447546e-08,5.010888e-08,0.600050,0.400090,-2.305915e-08,...,0.400090,1.884864e-07,1.645887e-07,1.021572e-07,0.954930,0.806964,-4.890708e-08,7.643711e-09,0.806964,0.000240
2017,SXS:BBH:2163,0.749995,0.249993,3.000059,0.599908,-1.924846e-08,8.244233e-08,0.599908,0.600011,1.990578e-08,...,0.600011,1.173319e-07,1.413648e-07,5.372690e-08,0.953784,0.811977,2.394728e-08,7.779425e-08,0.811977,0.000196
2018,SXS:BBH:2265,0.749997,0.249991,3.000099,0.000002,5.926665e-08,-3.818597e-08,0.000002,0.000005,6.723248e-08,...,0.000005,2.225474e-02,3.151077e-02,1.242202e-02,0.971102,0.540609,5.332686e-08,-7.735071e-08,0.540609,0.000069


In [3]:
## list of column names
column_names = list(df.columns)
print(column_names)

['waveform_id', 'mass1', 'mass2', 'mass_ratio', 'spin1', 'chi1x', 'chi1y', 'chi1z', 'spin2', 'chi2x', 'chi2y', 'chi2z', 'alpha', 'beta', 'gamma', 'massf', 'spinf', 'chifx', 'chify', 'chifz', 'eccentricity']


We select non-precessing binaries imposing the following criteria:
- if the spin magnitudes $||\vec\chi||<0.001$, the binary is classified as _non-spinning_;
- if the relative magnitude of the in-plane components $|\chi_{x,y}|/||\vec\chi||<0.001$, the binary is classified as (spinning) _non-precessing_

Out of the total 2019 binaries, 117 are non-spinning and 416 are non-precessing.

We also select non-eccentric binaries imposing that the eccentricity $e<0.001$, resulting in 1512 non-eccentric binaries.

Finally, we consider the subset of (spinning or non-spinning) non-precessing, non eccentric binaries, resulting in 394 configurations.

In [4]:
## select non-spinning binaries
mask_NS = (df['spin1']<1e-3)
mask_NS &= (df['spin2']<1e-3)
df2 = df[mask_NS]
df2['eccentricity'].shape

(117,)

In [5]:
## select spinning non-precessing binaries
mask_NP = ~mask_NS
mask_NP &= (df['chi1x'].abs()/df['spin1']<1e-3)
mask_NP &= (df['chi1y'].abs()/df['spin1']<1e-3)
mask_NP &= (df['chi2x'].abs()/df['spin2']<1e-3)
mask_NP &= (df['chi2y'].abs()/df['spin2']<1e-3)
df2 = df[mask_NP]
df2['eccentricity'].shape

(416,)

In [6]:
## select non-eccentric binaries
mask_NE = df['eccentricity']<1e-3
df2 = df[mask_NE]
df2['eccentricity'].shape

(1512,)

In [7]:
## all non-precessing non-eccentric binaries
mask = (mask_NS | mask_NP) & mask_NE
df2 = df[mask]
df2['eccentricity'].shape

(394,)

We will be interested in a subset of columns:

In [8]:
columns = ['waveform_id','mass1','mass2','mass_ratio','chi1z','chi2z','massf','chifz']
params = df[mask][columns]
params.head()

,waveform_id,mass1,mass2,mass_ratio,chi1z,chi2z,massf,chifz
0,SXS:BBH:0001,0.500000,0.500000,1.000000,1.209309e-07,1.221969e-07,0.951609,0.686462
1,SXS:BBH:0002,0.500000,0.500000,1.000000,9.484007e-08,9.314798e-08,0.951610,0.686448
6,SXS:BBH:0007,0.600000,0.400000,1.499999,8.644276e-08,1.553639e-07,0.955270,0.664091
13,SXS:BBH:0014,0.600272,0.400000,1.500679,-4.995200e-01,2.169143e-07,0.962685,0.540291
18,SXS:BBH:0019,0.600272,0.400191,1.499964,-4.995170e-01,4.994650e-01,0.959582,0.586755


We also renormalize mass1 and mass2 such that their sum is 1, because SXS waveforms are provided in units of total mass. A corresponding rescale of the total mass is also needed.

In [9]:
mtot = params['mass1'] + params['mass2']
params.loc[:,'mass1'] =  params.loc[:,'mass1']/ mtot
params.loc[:,'mass2'] =  params.loc[:,'mass2']/ mtot
params.loc[:,'massf'] =  params.loc[:,'massf']/ mtot
params.tail()

,waveform_id,mass1,mass2,mass_ratio,chi1z,chi2z,massf,chifz
2013,SXS:BBH:2159,0.749994,0.250006,2.999904,0.600059,-0.599733,0.959753,0.779858
2014,SXS:BBH:2160,0.750004,0.249996,3.000070,0.600044,-0.399866,0.958884,0.785484
2016,SXS:BBH:2162,0.750005,0.249995,3.000073,0.600050,0.400090,0.954915,0.806964
2017,SXS:BBH:2163,0.750004,0.249996,3.000059,0.599908,0.600011,0.953795,0.811977
2018,SXS:BBH:2265,0.750006,0.249994,3.000099,0.000002,0.000005,0.971115,0.540609


## Load NR fits

Next, we load NR fits for the amplitudes and phases of each waveform. The waveform are ordered through `waveform_id`, similarly to the metadata loaded above.

__Note that the amplitudes provided here are absolute, i.e. they are not relative to $A_{220}$. Similarly, phases are not relative to $\phi_{220}$.__

In [10]:
filename = '../postmerger/data/NR_fits/3dq8_20M_SXS_fits.pkl'
NR_fits = joblib.load(filename)
NR_fits.keys()

dict_keys(['time_from_peak', 'waveform_id', 'amps', 'phis', 'mismatch'])

You can access the multipoles $(l,m)$ and the corresponding modes that we fitted for:

In [11]:
LM = {lm:[] for lm in NR_fits['amps'].keys()}
for lm in LM.keys():
    modes_lm = list(NR_fits['amps'][lm].keys())
    LM[lm] = modes_lm
LM

{(2, 2): [(2, 2, 0), (2, 2, 1)],
 (2, 1): [(2, 1, 0), (2, 1, 1)],
 (3, 3): [(3, 3, 0), (3, 3, 1)],
 (3, 2): [(3, 2, 0), (3, 2, 1), (2, 2, 0), (2, 2, 1)],
 (4, 4): [(4, 4, 0), ((2, 2, 0), (2, 2, 0)), (4, 4, 1)],
 (4, 3): [(4, 3, 0), (4, 3, 1), (3, 3, 0), (3, 3, 1)],
 (5, 5): [(5, 5, 0), ((2, 2, 0), (3, 3, 0)), (5, 5, 1)],
 (2, -2): [(2, -2, 0), (2, -2, 1)],
 (2, -1): [(2, -1, 0), (2, -1, 1)],
 (3, -3): [(3, -3, 0), (3, -3, 1)],
 (3, -2): [(3, -2, 0), (3, -2, 1), (2, -2, 0), (2, -2, 1)],
 (4, -4): [(4, -4, 0), ((2, -2, 0), (2, -2, 0)), (4, -4, 1)],
 (4, -3): [(4, -3, 0), (3, -3, 0), (3, -3, 1)],
 (5, -5): [(5, -5, 0), ((2, -2, 0), (3, -3, 0)), (5, -5, 1)]}

Loading fits for the amplitudes is as easy as:

In [12]:
lm = (3,3)
mode = (3,3,0)
amp_fits = NR_fits['amps'][lm][mode]
print(amp_fits[:4])
print(amp_fits.shape)

[1.65043346e-05 1.65079495e-05 1.89956413e-02 1.93701060e-02]
(394,)


and similarly for the phases - they are defined in the interval $[0,2\pi]$:

In [13]:
lm = (3,3)
mode = (3,3,0)
phi_fits = NR_fits['phis'][lm][mode]
print(phi_fits[:4])
print(phi_fits.shape)

[4.21937387 4.21858389 6.28318531 2.04399969]
(394,)


We also provide the mismatches
$$
\mathcal{M}_{lm}=1-\frac{<h_{lm}^{\rm NR}|h_{lm}^{\rm fit}>}{\sqrt{<h_{lm}^{\rm NR}|h_{lm}^{\rm NR}><h_{lm}^{\rm fit}|h_{lm}^{\rm fit}>}}
$$
between numerical strains $h_{lm}^{\rm NR}$ and waveforms generated with the fitted values of amplitudes and phases $h_{lm}^{\rm fit}$.

In [14]:
lm = (3,3)
M_lm = NR_fits['mismatch'][lm]
print(M_lm[:4])

[0.84423992 0.7130507  0.00514694 0.04160523]
